# Projeto Final | Analytics Engineering
----
**Engenharia de Dados e Garantia de Qualidade no Conjunto de Dados do Airbnb no Rio de Janeiro**

# Etapa 1:

**Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**

   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

## Bibliotecas

In [1]:
from sqlalchemy import create_engine, text as sql_text
import pandas as pd
import datetime

import os

### Chamada da biblioteca para usar o SQL

In [2]:
%load_ext sql

## Informações do banco de dados

In [3]:
## Importar modulo de conexão e implementação do banco  
import modules.database as db

Estabelecer uma conexão com um banco de dados PostgreSQL

In [4]:
# Configurar conexão
engine = db.engine_db()

In [5]:
# Testar a conexão
try:
    with engine.connect():
        print("Conexão bem sucedida!")
except Exception as e:
    print("Erro ao conectar:", e)

Conexão bem sucedida!


## Camada Landing - Extração de dados

In [7]:
## Importar modulo para extração dos dados
from modules.etl import Landing

url = [
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/listings.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/calendar.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/reviews.csv.gz'
]
data_path = './data/'
landing_step = Landing(landing_path=data_path)
landing_step.extract(url=url)
landing_step.transform()

O diretório ./data/ não existe e tentaremos criar.
Diretório criado com sucesso!!
Efetuando a requisição para a url https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/listings.csv.gz
Efetuando a requisição para a url https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/calendar.csv.gz
Efetuando a requisição para a url https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/reviews.csv.gz
Salvando o arquivo ./data/listings.csv.gz
Salvando o arquivo ./data/calendar.csv.gz
Salvando o arquivo ./data/reviews.csv.gz


## Preparação do Datalake

In [8]:
import modules.database as db

db.criar_database()
db.criar_schemas()

Banco de dados airbnb_RJ criado com sucesso.
O Schema 'bronze' ja existe.
O Schema 'silver' ja existe.
O Schema 'gold' ja existe.


## Camada Bronze - Ingestão e Análise dos Dados

In [9]:
import pandas as pd
import modules.database as db

# Testar a conexão
try:
    print("Iniciando a estapa bronze.")

    ## Listings
    data_path = './data/'
    df_listings = pd.read_csv(data_path + 'listings.csv') # Carregando o csv listings em um Dataframe Python
    db.criar_tabela_df('bronze', 'listings', df_listings,'replace') # Criar e salvar o DataFrame na tabela "listings" dentro do esquema "bronze"

    ## Reviews
    df_reviews = pd.read_csv(data_path + 'reviews.csv') # Carregando o csv reviews em um Dataframe Python
    db.criar_tabela_df('bronze', 'reviews', df_reviews,'replace') # Criar e salvar o DataFrame na tabela "reviews" dentro do esquema "bronze"

    ## Calendar
    df_calendar = pd.read_csv(data_path + 'calendar.csv') # Carregando o csv calendar em um Dataframe Python
    db.criar_tabela_df('bronze', 'calendar', df_calendar,'replace') # Criar e salvar o DataFrame na tabela "calendar" dentro do esquema "bronze"

    print("Etapas bronze executada com sucesso!!")
except Exception as e:
    print("Ocorreu um erro inesperado ao efetua a etapa bronze:", e)

Iniciando a estapa bronze.
Criando a tabela 'listings' no schema 'bronze'....
A tabela 'listings' foi criada no schema 'bronze', e os dados foram inseridos.
Criando a tabela 'reviews' no schema 'bronze'....
A tabela 'reviews' foi criada no schema 'bronze', e os dados foram inseridos.
Criando a tabela 'calendar' no schema 'bronze'....


# Etapa 2:

**Data Clean - Camada Silver**

   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

## Camada Silver - Limpeza de Dados e Criação de colunas

### Leitura das tabelas bronze

In [ ]:
query = """
SELECT * FROM bronze.listings
"""
df_silver_listings = pd.read_sql(sql=sql_text(query), con=engine.connect())

In [ ]:
query = """
SELECT * FROM bronze.reviews
"""
df_silver_reviews = pd.read_sql(sql=sql_text(query), con=engine.connect())

In [54]:
query = """
SELECT * FROM bronze.calendar
"""
df_silver_calendar = pd.read_sql(sql=sql_text(query), con=engine.connect())

### Importação da biblioteca de limpeza

In [18]:
from modules.utils import data_profiling, remove_special_caracters, process_dataframe, remove_outliers, change_t_for_1

### Relatório com informações do perfil dos dados

In [ ]:
data_profiling(df_silver_listings, 'listings', 'data_profiling/')
# data_profiling(df_silver_reviews, 'reviews', 'data_profiling/')
# data_profiling(df_silver_calendar, 'calendar', 'data_profiling/')

### Processamento da tabela **listings**

##### Limpeza de caracteres

In [16]:
df_silver_listings["neighborhood_overview"] = df_silver_listings["neighborhood_overview"].apply(remove_special_caracters)
df_silver_listings["host_about"] = df_silver_listings["host_about"].apply(remove_special_caracters)

##### Identificar e lidar com valores ausentes, duplicatas e outliers

In [17]:
# colunas para considerar a eliminação de outliers
cols_listings = ['review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

df_process = process_dataframe(df_silver_listings, 'listings')
df_silver_listings = remove_outliers(df_process, cols_listings)

Analisando a tabela listings

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  listings : 
 ['license', 'calendar_updated', 'neighbourhood_group_cleansed', 'bathrooms', 'bedrooms', 'description']

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela listings

Removendo colunas constantes ...
Colunas constantes removidas da tabela listings: 
 ['scrape_id', 'amenities']

Quantidade de linhas (outliers) eliminadas:  27265


##### Conversão do T e F para 1 e 0

In [19]:
cols_listings2 = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']

for col in cols_listings2:
    df_silver_listings[col] = df_silver_listings[col].apply(change_t_for_1).astype("int")

##### Transformação da coluna bathroom_text em 2 colunas

In [20]:
df_silver_listings['bathrooms_text'].unique()

array(['1 bath', '1.5 baths', '5 baths', '2 baths', '7 baths', '3 baths',
       '1 shared bath', '4 baths', '2.5 baths', '5 shared baths',
       '1 private bath', '2.5 shared baths', '2 shared baths',
       '4.5 baths', '0 shared baths', 'Shared half-bath',
       '1.5 shared baths', '0 baths', None, '3.5 baths', '6 baths',
       '3 shared baths', '5.5 baths', '7.5 baths', '3.5 shared baths',
       '4.5 shared baths', '6.5 shared baths', '6.5 baths', '10 baths',
       '4 shared baths', '16 baths', '8 baths', 'Half-bath'], dtype=object)

In [21]:
# Preencha os valores nulos na coluna 'bathrooms_text' com '0 bath'
df_silver_listings['bathrooms_text'] = df_silver_listings['bathrooms_text'].fillna('0 bath')
    
# Defina o tipo de banheiro
df_silver_listings['bathroom_type'] = df_silver_listings['bathrooms_text'].str.replace(r'(\d+\.?\d*)', '', regex=True)
    
# Remova 's' do final de 'baths e traz o strip()'
df_silver_listings['bathroom_type'] = df_silver_listings['bathroom_type'].str.replace(r's?$', '', regex=True).apply(lambda x: x.strip())
    
# Extraia o número de banheiros usando expressões regulares
df_silver_listings['bathroom_quantity'] = df_silver_listings['bathrooms_text'].str.extract(r'(\d+\.?\d*)')[0].astype(float)
    
# Converta a coluna 'bathroom_quantity' para tipo numérico
df_silver_listings['bathroom_quantity'] = pd.to_numeric(df_silver_listings['bathroom_quantity'], errors='coerce').fillna(0)

In [22]:
df_silver_listings['bathroom_type'].unique()

array(['bath', 'shared bath', 'private bath', 'Shared half-bath',
       'Half-bath'], dtype=object)

In [23]:
df_silver_listings[["bathroom_type",'bathroom_quantity']]

,bathroom_type,bathroom_quantity
0,bath,1.0
1,bath,1.0
2,bath,1.5
3,bath,1.0
4,bath,1.0
...,...,...
32278,bath,1.0
32498,bath,1.0
32739,bath,2.0
33052,bath,1.0


### Processamento da tabela **reviews**

##### Limpeza de caracteres

In [29]:
df_silver_reviews["comments"] = df_silver_reviews["comments"].apply(remove_special_caracters)

##### Identificar e lidar com valores ausentes, duplicatas e outliers

In [32]:
# colunas para considerar a eliminação de outliers
# cols_reviews = []

df_process = process_dataframe(df_silver_reviews, 'reviews')
# df_silver_reviews = remove_outliers(df_process, cols_reviews)

### Processamento da tabela **calendar**

##### Limpeza de caracteres

In [56]:
df_silver_calendar["price"] = df_silver_calendar["price"].apply(remove_special_caracters).astype("float")

##### Identificar e lidar com valores ausentes, duplicatas e outliers

In [57]:
# colunas para considerar a eliminação de outliers
cols_calendar = ['price']

df_process = process_dataframe(df_silver_calendar, 'calendar')
df_silver_calendar = remove_outliers(df_process, cols_calendar)

##### Conversão do T e F para 1 e 0

In [58]:
cols_calendar2 = ['available']

for col in cols_calendar2:
    df_silver_calendar[col] = df_silver_calendar[col].apply(change_t_for_1).astype("int")

C:\Users\luana\AppData\Local\Temp\ipykernel_24688\3237782736.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver_calendar[col] = df_silver_calendar[col].apply(change_t_for_1).astype("int")


# Etapa 3:

**Data Quality - Camada Silver**

   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

### Função para verificar os casos nulos

In [63]:
def check_missing(df):
    res_missing = df.isna().sum()
    # res_missing = (res_missing/len(df))*100
    return res_missing[res_missing != 0]

### Função para verificar datas

In [86]:
def verificar_datas(df):

    ano_atual = pd.Timestamp.now().year

    # # Converter a coluna 'date' para o tipo datetime
    # df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # Verificar as condições
    condicao_dia = (df['date'].dt.day >= 1) & (df['date'].dt.day <= 31) # dias num intervalo de 1 a 31
    condicao_mes = (df['date'].dt.month >= 1) & (df['date'].dt.month <= 12) # meses num intervalo de 1 a 12
    condicao_ano = (df['date'].dt.year >= 1900) & (df['date'].dt.year <= ano_atual) # dias num intervalo de 1900 ao ano corrente

    # Aplicar as condições ao DataFrame
    df_filtrado = df[condicao_dia & condicao_mes & condicao_ano]

    # Verificar se há valores fora do intervalo
    valores_fora_intervalo = df[~(condicao_dia & condicao_mes & condicao_ano)]

    return df_filtrado, valores_fora_intervalo

### Processamento da tabela **listings**

In [ ]:
## Todas as colunas com dados faltantes 
check_missing(df_silver_listings).sort_values(ascending=False)

##### Preenchimento dos campos nulos com valores fixos
- df_cln['notRepairedDamage'] = df_cln['notRepairedDamage'].fillna("no_info")

##### Preenchimento dos campos nulos com o campo que mais se repete
- high_freq = df_cln['fuelType'].value_counts().idxmax()
- df_cln['fuelType'] = df_cln['fuelType'].fillna(high_freq)

##### Preenchimento dos campos nulos com valores fixos de outra coluna
- df_cln['model'] = df_cln['model'].fillna(df_cln['vehicleType'])

##### Padronização dos nomes e tipos das colunas

In [ ]:
# Definir o tipo das colunas que não serão de texto
list_int_listings = ['id', 'host_id', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']
list_datetime_listings = ['last_scraped']
list_float_listings = ['review_scores_location', 'review_scores_value', 'reviews_per_month']

In [ ]:
## Tipos das colunas

print('Integer')
print(df_silver_listings[list_int_listings].dtypes)

print('\nDatetime')
print(df_silver_listings[list_datetime_listings].dtypes)

print('\nFloat')
print(df_silver_listings[list_float_listings].dtypes)

In [ ]:
## Nessário apenas alterar o tipo das colunas 'last_scraped'
df_silver_listings['last_scraped'] = pd.to_datetime(df_silver_listings['last_scraped'], format='%Y-%m-%d')

In [ ]:
df_silver_listings[list_datetime_listings].dtypes

last_scraped    datetime64[ns]
dtype: object

### Processamento da tabela **reviews**

##### Padronização dos nomes e tipos das colunas

In [ ]:
# Definir o tipo das colunas que não serão de texto
list_int_reviews = ['listing_id', 'id', 'reviewer_id']
list_datetime_reviews = ['date']

In [ ]:
df_silver_reviews.dtypes

In [ ]:
## Apenas a alteração do tipo da coluna 'date' é necessária.
df_silver_reviews['date'] = pd.to_datetime(df_silver_reviews['date'], format='%Y-%m-%d')

In [ ]:
df_silver_reviews.dtypes

listing_id                int64
id                        int64
date             datetime64[ns]
reviewer_id               int64
reviewer_name            object
comments                 object
dtype: object

### Processamento da tabela **calendar**

##### Preenchimento dos campos nulos com o valor que mais se repete

In [88]:
## Colunas com dados faltantes + quant de dados faltantes
check_missing(df_silver_calendar).sort_values(ascending=False)

Series([], dtype: int64)

In [75]:
minimum_freq = df_silver_calendar['minimum_nights'].value_counts().idxmax()
maximum_freq = df_silver_calendar['maximum_nights'].value_counts().idxmax()

In [76]:
df_silver_calendar['minimum_nights'] = df_silver_calendar['minimum_nights'].fillna(minimum_freq)
df_silver_calendar['maximum_nights'] = df_silver_calendar['maximum_nights'].fillna(maximum_freq)

In [78]:
## Verificar novamente dados faltantes 
check_missing(df_silver_calendar).sort_values(ascending=False)

Series([], dtype: int64)

##### Padronização dos nomes e tipos das colunas

In [81]:
# Definir o tipo das colunas que não serão de texto
list_int_calendar = ['listing_id', 'minimum_nights', 'maximum_nights']
list_datetime_calendar = ['date']
list_float_calendar = ['price']

In [82]:
df_silver_calendar.dtypes

listing_id          int64
date               object
available           int32
price             float64
minimum_nights    float64
maximum_nights    float64
dtype: object

In [83]:
## Nessário apenas alterar o tipo das colunas 'date','minimum_nights', 'maximum_nights'

df_silver_calendar['date'] = pd.to_datetime(df_silver_calendar['date'], format='%Y-%m-%d')

for column in ['minimum_nights', 'maximum_nights']:
    #transforma a coluna para o tipo inteiro
    df_silver_calendar[column] = df_silver_calendar[column].astype("int")

In [84]:
## Verificar novamente tipos das colunas
df_silver_calendar.dtypes

listing_id                 int64
date              datetime64[ns]
available                  int32
price                    float64
minimum_nights             int32
maximum_nights             int32
dtype: object

In [85]:
df_silver_calendar['date']

0          2023-12-27
1          2023-12-28
2          2023-12-29
3          2023-12-30
4          2023-12-31
              ...    
13145590   2024-12-20
13145591   2024-12-21
13145592   2024-12-22
13145593   2024-12-23
13145594   2024-12-24
Name: date, Length: 12593670, dtype: datetime64[ns]

##### Verificação da precisão dos dados

In [90]:
## Verificação de datas 

df_filtrado, valores_fora_intervalo = verificar_datas(df_silver_calendar)

In [89]:
valores_fora_intervalo

,listing_id,date,available,price,minimum_nights,maximum_nights


In [94]:
## Verificação de preços  

df_silver_calendar[df_silver_calendar['price']<=0]

,listing_id,date,available,price,minimum_nights,maximum_nights


# Etapa 4:

**Testes de Qualidade - Camada Silver**

   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

# Etapa 5:

**Transformação de Dados com dbt - Camada Silver**

   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

# Etapa 6:

**Armazenamento de Dados em PostgreSQL - Camada Silver**

   - Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
   - Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

# Etapa 7:

**Validação de Expectativas com Great Expectations - Camada Silver**

   - Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
   - Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

# Etapa 8:

**Transformação de Dados com dbt - Camada Gold**

   - Utilize o dbt para criar a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "gold" e automatize a execução das transformações.
   - Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.